**2.7   Object Abstraction**

***2.7.1   String Conversion***

In [1]:
from datetime import date
tues = date(2019, 1, 1)


In [2]:
repr(tues)

'datetime.date(2019, 1, 1)'

In [3]:
str(tues)

'2019-01-01'

In [4]:
tues.__str__()

'2019-01-01'

In [5]:
tues.__repr__()

'datetime.date(2019, 1, 1)'

***2.7.2   Special Methods***

In [1]:
len('dasdwfgerf')

10

In [3]:
'dasdwfgerf'.__len__()

10

In [4]:
'dasdwfgerf'[3]

'd'

In [5]:
'dasdwfgerf'.__getitem__(3)

'd'

**Callable objects**. In Python, functions are first-class objects, so they can be passed around as data and have attributes like any other object. Python also allows us to define objects that can be "called" like functions by including a __call__ method. With this method, we can define a class that behaves like a higher-order function.

In [6]:
def make_adder(n):
    def adder(k):
        return n + k
    return adder

In [7]:
add_three = make_adder(3)
add_three(4)

7

In [8]:
add_three.__call__(4)

7

In [9]:
class adder():
    def __init__(self, n):
        self.n = n
    def __call__(self, k):
        return self.n + k
add_four = adder(4)

In [10]:
add_four(5)

9

Here, the Adder class behaves like the make_adder higher-order function, and the add_three_obj object behaves like the add_three function. We have further blurred the line between data and functions.

***2.7.3   Multiple Representations***

In [91]:

    
class Complex(Number):
    def add(self, other):
        return ComplexRI(self.real + other.real, self.imag + other.imag)
    def mul(self, other):
        magnitude = self.magnitude * other.magnitude
        return ComplexMA(magnitude, self.angle + other.angle)
    
from math import atan2
class ComplexRI(Complex):
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag
    @property
    def magnitude(self):
        return (self.real ** 2 + self.imag ** 2) ** 0.5
    @property
    def angle(self):
        return atan2(self.imag, self.real)
    def __repr__(self):
        return 'ComplexRI({0:g}, {1:g})'.format(self.real, self.imag)
from math import sin, cos, pi
class ComplexMA(Complex):
    def __init__(self, magnitude, angle):
        self.magnitude = magnitude
        self.angle = angle
    @property
    def real(self):
        return self.magnitude * cos(self.angle)
    @property
    def imag(self):
        return self.magnitude * sin(self.angle)
    def __repr__(self):
        return 'ComplexMA({0:g}, {1:g} * pi)'.format(self.magnitude, self.angle / pi)


In [92]:
ComplexRI(1, 2) 

ComplexRI(1, 2)

In [93]:
ri = ComplexRI(1, 2)

In [94]:
ri

ComplexRI(1, 2)

In [95]:
ri.magnitude

2.23606797749979

In [96]:
ComplexMA.type_tag = 'com'
ComplexRI.type_tag = 'com'

In [97]:
ma = ComplexMA(2, pi)
ma.angle = pi
ma.real

-2.0

Our implementation of complex numbers is now complete. Either class implementing complex numbers can be used for either argument in either arithmetic function in Complex.

In [98]:
from math import pi
ComplexRI(1,2) + ComplexMA(2, pi/2)

ComplexRI(1, 4)

In [99]:
ComplexRI(1,2) * ComplexMA(2, pi/2)

ComplexMA(4.47214, 0.852416 * pi)

**2.7.4 Generic Functions**

In [100]:
from math import gcd

In [101]:
class Rational(Number):
    def __init__(self, numer, denom):
        g = gcd(numer, denom)
        self.numer = numer // g
        self.denom = denom // g
    def __repr__(self):
        return 'Rational({0}, {1})'.format(self.numer, self.denom)
    def add(self, other):
        nx, dy = self.numer, self.denom
        ny, dx = other.numer, other.denom
        return Rational(nx * dx + ny * dy, dy * dx)
    def mul(self, other):
        return Rational(self.numer * other.numer, self.denom * other.denom)
    type_tag = 'rat'

In [102]:
Rational(1, 2) + Rational(1, 3)

Rational(5, 6)

In [103]:
c = ComplexRI(1, 2)
isinstance(c, Complex)

True

In [104]:
isinstance(c, ComplexMA)

False

In [105]:
def is_real(c):
    if isinstance(c, ComplexRI):
        return c.imag == 0
    elif isinstance(c, ComplexMA) and c.angle % pi == 0:
        return c.angle % pi == 0
    

In [106]:
is_real(ComplexRI(1, 0))


True

In [107]:
is_real(ComplexMA(1, pi))

True

In [108]:
Rational.type_tag = 'rat'
ComplexRI.type_tag = 'com'

In [109]:
Rational(1, 2).type_tag == Rational(1,3).type_tag

True

In [110]:
Rational(1, 2).type_tag == ComplexRI(1, 2).type_tag

False

In [111]:
def add_complex_and_rational(c, r):
    return ComplexRI(c.real + r.numer/r.denom, c.imag)

In [112]:
def mul_complex_and_rational(c, r):
    r_magnitude = r.numer / r.denom
    r_angle = 0
    if r_magnitude < 0:
        r_magnitude, r_angle = -r_magnitude, pi
    return ComplexMA(c.magnitude * r_magnitude, c.angle + r_angle)


In [113]:
def add_rational_and_complex(r, c):
    return add_complex_and_rational(c, r)
def mul_rational_and_complex(r, c):
    return mul_complex_and_rational(c, r)

In [115]:
class Number:
    def __add__(self, other):
        if self.type_tag == other.type_tag:
            return self.add(other)
        elif (self.type_tag, other.type_tag) in self.adders:
            return self.cross_apply(other, self.adders)
    def __mul__(self, other):
        if self.type_tag == other.type_tag:
            return self.mul(other)
        elif (self.type_tag, other.type_tag) in self.multipliers:
            return self.cross_apply(other, self.multipliers)
    def cross_apply(self, other, cross_fns):
        cross_fn = cross_fns[(self.type_tag, other.type_tag)]
        return cross_fn(self, other)
    adders = {('com', 'rat'): add_complex_and_rational, ( 'rat', 'com'): add_rational_and_complex}
    multipliers = {('com', 'rat'): mul_complex_and_rational, ('rat', 'com'): mul_rational_and_complex}
 

In [116]:
ComplexRI(1, 2) + Rational(1, 2)

ComplexRI(1.5, 2)

In [117]:
Rational(1, 2) * ComplexRI(1, pi/2)

ComplexMA(0.931048, 0.319546 * pi)

**Coercion.** In the general situation of completely unrelated operations acting on completely unrelated types, implementing explicit cross-type operations, cumbersome though it may be, is the best that one can hope for. Fortunately, we can sometimes do better by taking advantage of additional structure that may be latent in our type system. Often the different data types are not completely independent, and there may be ways by which objects of one type may be viewed as being of another type. This process is called coercion. For example, if we are asked to arithmetically combine a rational number with a complex number, we can view the rational number as a complex number whose imaginary part is zero. After doing so, we can use Complex.add and Complex.mul to combine them.

In [118]:
def rational_to_complex(r):
    return ComplexRI(r.numer, r.denom)

In [ ]:
class Number:
    def __add__(self, other):
        x, y = self.coerce(other)
        return x.add(y)
    def __mul__(self, other):
        x, y = self.coerce(other)
        return x.mul(y)
    def coerce(self, other):
        if self.type_tag == other.type_tag:
            return self, other
        elif (self.type_tag, other.type_tag) in self.coercions:
            return (self.coerce_to(other), other)
        elif (other.type_tag, self.type_tag) in self.coercions:
            return (self, other.coerce_to(self.tag_type))
    def coerce_to(self, other_tag):
        coercion_fn = self.coercions[(self.type_tag, other_tag)]
        return coercion_fn(self)
    coercions = {( 'rat', 'com'): rational_to_complex}